### Download GEDI data in VS Code
#### B.A. Wilder
#### 06/14/21
#### link to tutorial = https://lpdaac.usgs.gov/resources/e-learning/spatial-querying-of-gedi-version-2-data-in-python/

In [1]:
# Import packages
import requests as r
from datetime import datetime
import os

In [2]:
# Define a function called 'gedi_finder' that takes in product and bbox as inputs,
# and outputs a list of links to download intersecting GEDI granules
def gedi_finder(product,bbox):
    """
    Inputs are 1) GEDI product and 2) bounding box.
    
    1) products include:
    GEDI01_B.002 = GEDI L1B Geolocated Waveform Data Global Footprint Level 
    GEDI02_A.002 = GEDI L2A Elevation and Height Metrics Data Global Footprint Level
    GEDI02_B.002 = GEDI L2B Canopy Cover and Vertical Profile Metrics Data Global Footprint Level

    2) bounding box coordinates in LL Longitude, LL Latitude, UR Longitude, UR Latitude format

    
    Output includes list of links to download GEDI granules

    """
    # Define the base url ("CMR") 
    cmr = "https://cmr.earthdata.nasa.gov/search/granules.json?pretty=true&provider=LPDAAC_ECS&page_size=2000&concept_id="
    
    # Set up dictionary where key is GEDI shortname + version
    concept_ids = {'GEDI01_B.002': 'C1908344278-LPDAAC_ECS', 
                   'GEDI02_A.002': 'C1908348134-LPDAAC_ECS', 
                   'GEDI02_B.002': 'C1908350066-LPDAAC_ECS'}
    
    # CMR uses pagination for queries with more features returned than the page size
    page = 1
    bbox = bbox.replace(' ', '')  # remove any white spaces
    try:
        # Send GET request to CMR granule with inputs
        cmr_response = r.get(f"{cmr}{concept_ids[product]}&bounding_box={bbox}&pageNum={page}").json()['feed']['entry']

        # If 2000 features are returned (max), move to the next page and submit another request, and append
        while len(cmr_response) % 2000 == 0:
            page += 1
            cmr_response += r.get(f"{cmr}{concept_ids[product]}&bounding_box={bbox}&pageNum={page}").json()['feed']['entry']
        
        # CMR returns more info than just Data Pool links. Use list comprehension to return a list of DP links
        return [c['links'][0]['href'] for c in cmr_response]
    except:
        # If request fails ...
        print(r.get(f"{cmr}{concept_ids[product]}&bounding_box={bbox.replace(' ', '')}&pageNum={page}").json())

# Display help
help(gedi_finder)

Help on function gedi_finder in module __main__:

gedi_finder(product, bbox)
    Inputs are 1) GEDI product and 2) bounding box.
    
    1) products include:
    GEDI01_B.002 = GEDI L1B Geolocated Waveform Data Global Footprint Level 
    GEDI02_A.002 = GEDI L2A Elevation and Height Metrics Data Global Footprint Level
    GEDI02_B.002 = GEDI L2B Canopy Cover and Vertical Profile Metrics Data Global Footprint Level
    
    2) bounding box coordinates in LL Longitude, LL Latitude, UR Longitude, UR Latitude format
    
    
    Output includes list of links to download GEDI granules



In [3]:
# Provide inputs here
# For this example, I'm going to ask for canopy data for a small area near Yosemite
product = 'GEDI02_A.002'
bbox = '-119.89452770513797,37.526329989702994,-119.57868057779892,37.72375321521001'

In [4]:
# Call gedi_finder function and print the amount of granules found
granules = gedi_finder(product, bbox)
print(f"{len(granules)} {product} Version 2 granules found.")

16 GEDI02_A.002 Version 2 granules found.


In [5]:
# Export list to a .txt file using the current datetime as name
outName = f"{product.replace('.', '_')}_GranuleList_{datetime.now().strftime('%Y%m%d%H%M%S')}.txt"
with open(outName, 'w') as gf:
    for g in granules:
        gf.write(f"{g}\n")  # \n = new line
print(f"File containing links to intersecting {product} Version 2 data has been saved to:\n {os.getcwd()}\{outName}")

File containing links to intersecting GEDI02_A.002 Version 2 data has been saved to:
 /home/brent/Documents/Code/GEDI_Test\GEDI02_A_002_GranuleList_20210614173542.txt


### Once txt file has been generated you can now use the DAACDataDownload python script in terminal window
### 1) cd into the current directory
### 2) copy and paste this line with the correct .txt file

#### **_python DAACDataDownload.py -dir Downloads -f GEDI02_B_002_GranuleList_20210614151620.txt_**

### 3) you will be prompted to enter your id and password for NASA Earthdata
